In [5]:


import os

In [6]:
%pwd

'c:\\Users\\abhay\\Desktop\\ML Project\\End-to-End-ML-Project-\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\abhay\\Desktop\\ML Project\\End-to-End-ML-Project-'

In [27]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str 

In [28]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [29]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMNS

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
        )

        return model_evaluation_config
        

In [30]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [31]:
class ModelEvaluation:
    def __init__( self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_matrics(self, actual, pred):
        rmse = np.sqrt(mean_absolute_error(actual, pred))
        mae  = mean_absolute_error(actual, pred)
        r2   = r2_score(actual, pred)

        return rmse, mae, r2
    
    def prediction(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        predicted_qualities  = model.predict(test_x)

        (rmse, mae, r2) = self.eval_matrics(test_y, predicted_qualities)

        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path = Path(self.config.metric_file_name), data = scores)
        

In [32]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.prediction()

except Exception as e:
    raise e

[2025-06-19 19:54:59,450: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-19 19:54:59,469: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-19 19:54:59,472: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-19 19:54:59,475: INFO: common: created directory at: artifacts]
[2025-06-19 19:54:59,478: INFO: common: created directory at: artifacts/model_evaluation]
[2025-06-19 19:54:59,491: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
